# Privacy Policies: Decision Tree Classifier

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ssl
import nltk
import os.path
import re
import collections
import pandas as pd
import seaborn as sns
import socket
import pydotplus
import random
import logging

# from six import StringIO

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from langdetect import detect
from newspaper import Article
from bs4 import BeautifulSoup, NavigableString, Tag
from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError
from http.client import IncompleteRead
from IPython.display import Image
from pprint import pprint
from time import time

stop_words = stopwords.words('english')

### Methods

Method that converts list to string

In [2]:
def list_to_string(s):
	# initialize an empty string
	str1 = ""
	
	# traverse in the string
	for ele in s:
		str1 += ele
	
	# return string
	return str1

Method to scrape a privacy policy from a given URL

In [3]:
def scrape_policy_url(policy_url):
	parsed_policy = ""
	try:
		article = Article(policy_url)
		article.download()  # Downloads the link’s HTML content
		article.parse()  # Parse the article
		article.nlp()
		# print(policy_url)
		# print(article.text)
		parsed_policy = article
	# except article.ArticleException as ae:
	# print(ae)
	except:
		pass
	# avoid too many requests error from Google
	
	return parsed_policy

Method to find the relevant paragraph of a privacy policy

In [4]:
def find_relevant_paragraph(policy_url):
	# policy_url = "https://irs.gov/privacy-disclosure/report-phishing"
	
	pre_dataframe = [[]]
	potential_paragraph = ""
	headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) "
	                         "Chrome/41.0.2228.0 Safari/537.3"}
	
	# header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) Gecko/20100101 Firefox/32.0', }
	
	try:
		req = Request(policy_url, headers=headers)
		html = urlopen(req, timeout=1)
		
		# fill first element of the list (first cell of row) with the relevant website title
		# policy_list.append(urlparse(policy.url).netloc + ".txt")
		# html = urlopen('https://automattic.com/privacy/')
		
		bs = BeautifulSoup(html, "html.parser")
		# bs = BeautifulSoup(req.text, "lxml")
		
		# 6 levels of HTML-headers
		# titles = bs.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
		# print('List all the header tags :', *titles, sep='\n\n')
		
		for header in bs.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
			# print(header.text.strip())
			if ("right" or "rights") in header.text.strip().lower():
				
				nextNode = header
				while True:
					nextNode = nextNode.nextSibling
					if nextNode is None:
						break
					if isinstance(nextNode, NavigableString):
						# print(nextNode.strip())
						potential_paragraph = potential_paragraph + " " + nextNode.strip()
					if isinstance(nextNode, Tag):
						if nextNode.name == "h2":
							break
						# print(nextNode.get_text(strip=True).strip())
						potential_paragraph = potential_paragraph + " " + nextNode.get_text(strip=True).strip()
	
	# policy_list.append(header.text.strip())
	# print("*** *** *** END")
	# pre_dataframe.append(policy_list)
	except HTTPError as e:
		# do something
		print('Error code: ', e.code)
		pass
	except URLError as e:
		# do something
		print('Reason: ', e.reason)
		pass
	except socket.timeout as e:
		# if urlopen takes too long just skip the url
		pass
	except IncompleteRead:
		# Oh well, reconnect and keep trucking
		pass
	except ConnectionResetError:
		pass
	except ssl.SSLWantReadError:
		pass
	
	return potential_paragraph

Method to read policy texts from specified directory. The language and length of the policy is held against a check to filter out irrelevant policies

In [5]:
def read_texts(n_words_policy, check_reduced):
	# print("Extracting texts from: " + data_dir)
	all_files = os.listdir(data_dir)
	selected_files = []
	policies_list = []
	titles = []
	
	# filter on txt files (redundant, but safety first)
	txt_files = filter(lambda x: x[-4:] == '.txt', all_files)
	# print("Potential policies found: " + str(len(all_files)))
	
	for file in txt_files:
		# print(file)
		new_file = []
		potential_policy_text = ""
		with open((data_dir + "\\" + file), "r", encoding="utf8") as policy:
			data = policy.readlines()
			policy_text = list_to_string(data[5:])
			policy_url = list_to_string(data[1])[5:]
			new_file[:5] = data[:5]
			if len(policy_text.split()) > n_words_policy:
				lang = detect(policy_text)
				if lang == 'en':
					# print("het is engels")
					# print(lang)
					
					# -if text can be reduced, reduce
					if check_reduced:
						potential_policy_text = find_relevant_paragraph(policy_url)
					
					if len(potential_policy_text) > 20:
						# print("Considering... reduced text")
						policies_list.append(potential_policy_text)
						new_file.append("Reduced Policy: " + potential_policy_text)
					else:
						# print("Considering... full text")
						# otherwise consider text in full
						policies_list.append(policy_text)
					title = (data[0].rstrip("\n").replace("Title: ", ""))
					titles.append(title)
					selected_files.append(file)
				else:
					# print("het is geen engels")
					# print(lang)
					pass
		if len(potential_policy_text) > 20:
			with open((data_dir + "\\" + file), "w", encoding="utf8") as selected_file:
				selected_file.writelines(new_file)
	
	print("Final number of policies after filtering: " + str(len(policies_list)))
	# return policies_list, titles
	return selected_files, policies_list

Method to link the read policies to the URLs of the dataframe 

In [6]:
def select_policies(url_list, policy_list, policy_list_text):
	url_policy_list = []
	counter = 0
	for url in url_list:
		url_counter = 0
		for i, policy in enumerate(policy_list, start=0):  # default is zero
			if url in policy:
				if url_counter == 0:
					url_policy_list.append(policy_list_text[i])
					# print(url)
					# print(policy)
				else:
					# print("Dubbel")
					# if there are multiple policies for one website, select the one that is longer
					if len(policy_list_text[i]) > len(url_policy_list[-1]):
						url_policy_list[-1] = policy_list_text[i]
				url_counter = 1
		if url_counter == 0:
			url_policy_list.append("")
	
	return url_policy_list

Read labelled dataset into a pandas dataframe

In [7]:
def read_data():
	df_pp_read = pd.read_excel(dir)
	
	# SELECT ONLY URLS THAT ARE EITHER RELEVANT OR NOT RELEVANT WITH EXCLUSION CRITERIA CODE E1 (NOT RELEVANT)
	df_pp_relevant = df_pp_read[(df_pp_read['included'] == 'y') | (df_pp_read['included'] == 'e1')]
	
	df_pp_relevant_scoped = df_pp_relevant[
		['URL', 'included', 'UR_explicitly_mentioned', 'access', 'rectification', 'erasure', 'restriction',
		 'data_portability', 'object', 'automated_decision_making']]
	
	df_user_rights = df_pp_relevant[['access', 'rectification', 'erasure', 'restriction',
	                                 'data_portability', 'object', 'automated_decision_making']]
	
	# plot_stats(df_user_rights)
	
	# LIST OF URLs
	url_list = df_pp_relevant_scoped['URL'].tolist()
	
	return df_pp_read, df_pp_relevant, df_pp_relevant_scoped, df_user_rights, url_list

Identify document features (BOG approach)

In [8]:
def document_features2(document):
	document_words = set(document)  # faster than list
	features = {}
	for word in word_features_p:
		features['contains({})'.format(word)] = (word in document_words)
	return features

Preprocess policies

In [9]:
def pre_processing(policies_list):
	counter = 0
	lemma = WordNetLemmatizer()
	# Remove punctuation
	# print("removing punctuation ...")
	policies_list = [re.sub('[,\.!?&“”():*;"]', '', x) for x in policies_list]
	policies_list = [re.sub('[\'’/]', '', x) for x in policies_list]
	
	# Convert the titles to lowercase
	policies_list = [x.lower() for x in policies_list]
	
	# remove short words
	policies_list = [' '.join([w for w in x.split() if len(w) > 3]) for x in policies_list]
	
	# tokenizing
	tokenized_policies = [x.split() for x in policies_list]
	
	# lemmatize
	tokenized_policies = [[lemma.lemmatize(y) for y in x] for x in tokenized_policies]
	
	# stemming
	porter = PorterStemmer()
	tokenized_policies = [[porter.stem(y) for y in x] for x in tokenized_policies]
	
	# remove emailadresses and URLs
	# tokenized_policies = [[y for y in x if("@" not in y)] for x in tokenized_policies]
	tokenized_policies = [[y for y in x if (("@" not in y) and ("http" not in y) and
	                                        ("wwww" not in y) and ("com" not in y))] for x in tokenized_policies]
	
	
	# remove stopwords
	tokenized_policies = [[y for y in x if not y in stop_words] for x in tokenized_policies]
	
	# print(relevant_words_lemmatized)
	# print(tokenized_policies)
	# print("length of policy: " + str(len(tokenized_policies)))
	# print("\n")
	
	detokenized_policies = []
	for i in range(len(policies_list)):
		t = ' '.join(tokenized_policies[i])
		detokenized_policies.append(t)
		
	return detokenized_policies

### Main

Directories

In [10]:
	pd.set_option("display.max_rows", None, "display.max_columns", None)  # print all values of dataframe
	dir = "data/PP_comparison_classification.xlsx"
	data_dir = "data/privacy_policies"

Read the separate datasets and link them

In [ ]:
	print("Reading data from: " + dir + " ...")
	df_pp_read, df_pp_relevant, df_pp_relevant_scoped, df_user_rights, url_list = read_data()
	
	# search for corresponding policies
	print("Extracting policy texts from: " + data_dir + " ...")
	policy_list, policy_list_text = read_texts(140, False)
	
	print("Link dataset to extracted policies ... \n")
	url_list_policies = select_policies(url_list, policy_list, policy_list_text)
	url_list_policies = pre_processing(url_list_policies)

Combine datasets in one dataframe

In [ ]:
	df_pp_relevant_scoped.insert(2, 'policy', url_list_policies)

Print empty policies

In [ ]:
#print(df_pp_relevant_scoped[(df_pp_relevant_scoped['policy'] == "")])

#### Policy distribution

In [ ]:
	categories = ['UR_explicitly_mentioned', 'access', 'rectification', 'erasure', 'restriction',
	              'data_portability', 'object', 'automated_decision_making']
	
	# what is the policy distribution?
	df_pp_filtered = df_pp_relevant_scoped[(df_pp_relevant_scoped['policy'] != "")]
	print("Policy distribution after filtering out empty policy texts")
	print("* Number of policies included with label \"y\": " + str(len(df_pp_filtered[(df_pp_filtered['included'] == 'y')])))
	print("* Number of policies excluded with label \"e1\": " + str(len(df_pp_filtered[(df_pp_filtered['included'] == 'e1')])))
	print("Total number of policies: " + str(len(df_pp_filtered)) + "\n")
	
	for category in categories:
		print("Category {}".format(category))
		print("Positive labels: {}".format(len(df_pp_filtered[(df_pp_filtered[category] == 1)])))
		print("Negative labels: {} \n".format(len(df_pp_filtered[(df_pp_filtered[category] == 0)])))

	df_pp = df_pp_filtered

#### Write processed dataframe to file (attempt to save memory)

In [10]:
df_pp.to_csv('data/processed_policies.csv')

NameError: name 'df_pp' is not defined

In [ ]:
#### Read processed dataframe

In [11]:
df_pp = pd.read_csv("data/processed_policies.csv")

#### Create dictionary consisting of {policy, label}

In [14]:
# policy_dict = [(policy_t.split(), label)
# 	             for policy_t in df_pp['policy'].to_list()
# 	             for label in df_pp['data_portability'].to_list()]
	

# # random.shuffle(documents)
# random.shuffle(policy_dict)

##### Tokenize and compute most frequent words (BOW)

In [15]:
	# BAG OF WORDS APPROACH
	# # To limit the number of features that the classifier needs to process, we begin by constructing a list of the 2000
	# # most frequent words in the overall corpus [1]. We can then define a feature extractor [2] that simply checks whether
	# # each of these words is present in a given document.
	tokenized_policies2 = [x.split() for x in df_pp['policy'].to_list()]
	#flatten
	tokenized_policies_f = [word for policy in tokenized_policies2 for word in policy]
	all_words_p = nltk.FreqDist(w.lower() for w in tokenized_policies_f)
# 	all_words_p.most_common(150)
	print("Number of words in the corpus after preprocessing: {}".format(len(tokenized_policies_f)))
	print("Number of unique words after preprocessing: {}".format(len(all_words_p)))

Number of words in the corpus after preprocessing: 214047
Number of unique words after preprocessing: 6165


##### Extract features:  based on the most frequent words list [top-100 due to memory issues]

In [20]:
	word_features_p = list(all_words_p)[:200]
	featuresets_p = [(document_features2(d), c) for (d, c) in policy_dict]

MemoryError: 

#### Explore policy length distribution

In [ ]:
RTobject =df_pp[df_pp['object'] == 1]['policy'].str.len()
sns.distplot(RTobject, label='Right to Object')
NRTobject = df_pp[df_pp['object'] == 0]['policy'].str.len()
sns.distplot(NRTobject, label='No GDPR-founded Right to Object')
plt.title('Distribution by Length')
plt.legend()

##### Divide data into training and testset

In [11]:
# size_p = int(len(featuresets_p) * 0.5)
# train_set_p, test_set_p = featuresets_p[size_p:], featuresets_p[:size_p]
X_train, X_test, y_train, y_test = train_test_split(df_pp['policy'], df_pp['object'], test_size=0.3, random_state=0)

#### Train classifier and classify

In [18]:
# classifier = nltk.DecisionTreeClassifier.train(train_set_p)
# print(nltk.classify.accuracy(classifier, test_set_p))
# print(classifier.pseudocode(depth=4))

0.5054283164782868
if contains(inform) == False: 
  if contains(servic) == False: 
    if contains(manag) == False: return '1'
    if contains(manag) == True: return '0'
  if contains(servic) == True: return '0'
if contains(inform) == True: 
  if contains(person) == False: 
    if contains(delet) == False: return '1'
    if contains(delet) == True: return '0'
  if contains(person) == True: return '0'



In [ ]:
# accuracy = nltk.classify.accuracy(nltk_model, test_set_p)*100
# print("{} Accuracy: {}".format("Decision Tree", accuracy))

## Countvectorizer Approach

##### Search for best variables (countvectorizer)

In [ ]:
tfidf_pipeline = Pipeline([
    ('cv', CountVectorizer()),
    ('dt', DecisionTreeClassifier()),
])

# increase processing time in a combinatorial way
parameters = {
    'cv__min_df': (0.5, 0.75, 1.0),
#     'tfidf__max_df': (0.5, 0.75, 1.0),
    'cv__max_features': (None, 5000, 10000, 50000),
    'cv__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3))  # unigrams or bigrams
}
        
grid_search = GridSearchCV(tfidf_pipeline, parameters)

print("Performing grid search...")
print("tf-idf pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(X_train, y_train) 
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

#### Countvectorizer

In [47]:
# Instantiate the vectorizer
count_vectorizer = CountVectorizer(analyzer='word', 
                            ngram_range=(2, 4), max_df=0.7, min_df=1, max_features=None)

# fit and transform on it the training features
count_vectorizer.fit(X_train)
X_train_word_features = count_vectorizer.transform(X_train)

#transform the test features to sparse matrix
test_features = count_vectorizer.transform(X_test)

classifier = DecisionTreeClassifier(criterion="gini")

classifier.fit(X_train_word_features, y_train)
y_pred = classifier.predict(test_features)
y_pred_prob = classifier.predict_proba(test_features)[:, 1]

print('CV Accuracy: {}'.format(accuracy_score(y_test, y_pred)))
print("CV ROC_AUC Score: {}\n".format(roc_auc_score(y_test, y_pred_prob)))

print(classification_report(y_test, y_pred))

print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))
print(pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = [['actual', 'actual'], ['Object', 'not-Object']],
    columns = [['predicted', 'predicted'], ['Object', 'not-Object']])) 

CV Accuracy: 0.76
CV ROC_AUC Score: 0.76

              precision    recall  f1-score   support

           0       0.78      0.72      0.75        25
           1       0.74      0.80      0.77        25

    accuracy                           0.76        50
   macro avg       0.76      0.76      0.76        50
weighted avg       0.76      0.76      0.76        50

              precision    recall  f1-score   support

           0       0.78      0.72      0.75        25
           1       0.74      0.80      0.77        25

    accuracy                           0.76        50
   macro avg       0.76      0.76      0.76        50
weighted avg       0.76      0.76      0.76        50

                  predicted           
                     Object not-Object
actual Object            18          7
       not-Object         5         20


#### Visualize Countvector Decision-Tree Classification Results

In [ ]:
# Visualize data
dot_data = export_graphviz(classifier,
                                feature_names=count_vectorizer.get_feature_names(),
                                out_file=None,
                                filled=True,
                                rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)

colors = ('turquoise', 'orange')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])

graph.write_png('output/cv-tree.png')

## TD-IDF Approach

##### Search for best variables (tf-idf)

In [44]:
tfidf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('dt', DecisionTreeClassifier()),
])

# increase processing time in a combinatorial way
parameters = {
    'tfidf__min_df': (0.5, 0.75, 1.0),
#     'tfidf__max_df': (0.5, 0.75, 1.0),
    'tfidf__max_features': (None, 5000, 10000, 50000),
    'tfidf__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3))  # unigrams or bigrams
}
        
grid_search = GridSearchCV(tfidf_pipeline, parameters)

print("Performing grid search...")
print("tf-idf pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(X_train, y_train) 
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
tf-idf pipeline: ['vect', 'tfidf', 'dt']
parameters:
{'tfidf__max_features': (None, 5000, 10000, 50000),
 'tfidf__min_df': (0.5, 0.75, 1.0),
 'tfidf__ngram_range': ((1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3))}


c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:

c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:

c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:

c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\users\aaberkan\onedrive - ugent\scripts\nlp4re-privacy-policies\nlp4re-pp-classification\venv\lib\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:

done in 403.803s

Best score: 0.729
Best parameters set:
	tfidf__max_features: 50000
	tfidf__min_df: 0.75
	tfidf__ngram_range: (1, 1)


#### TF-IDF Vectorizer

In [51]:
# Instantiate the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# fit and transform on it the training features
tfidf_vectorizer.fit(X_train)
X_train_word_features = tfidf_vectorizer.transform(X_train)

#transform the test features to sparse matrix
test_features = tfidf_vectorizer.transform(X_test)

tdidf_dt_classifier = DecisionTreeClassifier(criterion="gini")

tdidf_dt_classifier.fit(X_train_word_features, y_train)
y_pred = tdidf_dt_classifier.predict(test_features)
y_pred_prob = tdidf_dt_classifier.predict_proba(test_features)[:, 1]

print('CV Accuracy: {}'.format(accuracy_score(y_test, y_pred)))
print("CV ROC_AUC Score: {}\n".format(roc_auc_score(y_test, y_pred_prob)))

print(classification_report(y_test, y_pred))

print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))
print(pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = [['actual', 'actual'], ['Object', 'not-Object']],
    columns = [['predicted', 'predicted'], ['Object', 'not-Object']])) 

CV Accuracy: 0.72
CV ROC_AUC Score: 0.7120000000000002

              precision    recall  f1-score   support

           0       0.68      0.84      0.75        25
           1       0.79      0.60      0.68        25

    accuracy                           0.72        50
   macro avg       0.73      0.72      0.72        50
weighted avg       0.73      0.72      0.72        50

              precision    recall  f1-score   support

           0       0.68      0.84      0.75        25
           1       0.79      0.60      0.68        25

    accuracy                           0.72        50
   macro avg       0.73      0.72      0.72        50
weighted avg       0.73      0.72      0.72        50

                  predicted           
                     Object not-Object
actual Object            21          4
       not-Object        10         15


#### Visualize TD-IDF Decision-Tree Classification Results

In [25]:
# Visualize data
dot_data = export_graphviz(tdidf_dt_classifier,
                                feature_names=word_vectorizer.get_feature_names(),
                                out_file=None,
                                filled=True,
                                rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)

colors = ('turquoise', 'orange')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])

graph.write_png('output/tfidf-tree.png')

True